# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [58]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import warnings
import re
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
warnings.simplefilter('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [59]:
# load data from database
engine = create_engine('sqlite:///disaster_response.db')
df = pd.read_sql_table('disaster_response', engine)

In [60]:
df.head(3)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26216 entries, 0 to 26215
Data columns (total 40 columns):
id                        26216 non-null int64
message                   26216 non-null object
original                  10170 non-null object
genre                     26216 non-null object
related                   26216 non-null int64
request                   26216 non-null int64
offer                     26216 non-null int64
aid_related               26216 non-null int64
medical_help              26216 non-null int64
medical_products          26216 non-null int64
search_and_rescue         26216 non-null int64
security                  26216 non-null int64
military                  26216 non-null int64
child_alone               26216 non-null int64
water                     26216 non-null int64
food                      26216 non-null int64
shelter                   26216 non-null int64
clothing                  26216 non-null int64
money                     26216 non-null i

In [62]:
df.shape

(26216, 40)

In [63]:
X = df['message']
y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

In [64]:
X[:10]

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
5               Information about the National Palace-
6                       Storm at sacred heart of jesus
7    Please, we need tents and water. We are in Sil...
8      I would like to receive the messages, thank you
9    I am in Croix-des-Bouquets. We have health iss...
Name: message, dtype: object

In [65]:
y[:10]

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [66]:
X.shape

(26216,)

In [67]:
y.shape

(26216, 36)

### 2. Write a tokenization function to process your text data

In [68]:
def tokenize(text):
    '''
    Tokenization function to process data.
    
    Parameters:
    text (pandas Series): text to be processed
    
    Returns:
    clean_tokens (list): list of words converted to a lower case with removed stop words
    '''
    # Converting everything to lower case
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize words
    tokens = word_tokenize(text)
    
    # normalization word tokens and remove stop words
    normlizer = PorterStemmer()
    stop_words = stopwords.words("english")
    
    clean_tokens = [normlizer.stem(word) for word in tokens if word not in stop_words]
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [69]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [70]:
%%time
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 777)

model_initial = pipeline.fit(X_train, y_train)

CPU times: user 1min 27s, sys: 515 ms, total: 1min 28s
Wall time: 1min 29s


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [71]:
def eval_metrics(real_labels, predicted_labels, col_names):
    '''
    Evalute ML pipeline model's performance.
    
    Parameters:
    real_labels (list): real labels
    predicted_labels (list): predicted labels
    col_names (list): list of strings containing column names.
       
    Returns:
    performance_metrics (pandas DataFrame): accuracy, precision, recall and f1 score
    '''
    
    metrics = []
    
    # Evaluate metrics for each set of labels
    for i in range(len(col_names)):
        accuracy = accuracy_score(real_labels[:, i], predicted_labels[:, i])
        precision = precision_score(real_labels[:, i], predicted_labels[:, i], average='micro')
        recall = recall_score(real_labels[:, i], predicted_labels[:, i], average='micro')
        f1 = f1_score(real_labels[:, i], predicted_labels[:, i], average='micro')
        
        metrics.append([accuracy, precision, recall, f1])
    
    # store metrics
    metrics = np.array(metrics)
    performance_metrics = pd.DataFrame(data = metrics, index = col_names, columns = ['Accuracy', 'Precision', 'Recall', 'F1'])
      
    return performance_metrics

In [72]:
%%time
# Calculate evaluation metrics for test set
y_test_pred = pipeline.predict(X_test)
col_names = list(y.columns.values)

eval_metrics_test = eval_metrics(np.array(y_test), y_test_pred, col_names)

CPU times: user 9.23 s, sys: 172 ms, total: 9.4 s
Wall time: 9.44 s


In [73]:
eval_metrics_test

,Accuracy,Precision,Recall,F1
related,0.810192,0.810192,0.810192,0.810192
request,0.882667,0.882667,0.882667,0.882667
offer,0.994507,0.994507,0.994507,0.994507
aid_related,0.746262,0.746262,0.746262,0.746262
medical_help,0.924168,0.924168,0.924168,0.924168
medical_products,0.948276,0.948276,0.948276,0.948276
search_and_rescue,0.973146,0.973146,0.973146,0.973146
security,0.981080,0.981080,0.981080,0.981080
military,0.967043,0.967043,0.967043,0.967043
child_alone,1.000000,1.000000,1.000000,1.000000


In [74]:
eval_metrics_test.describe()

,Accuracy,Precision,Recall,F1
count,36.000000,36.000000,36.000000,36.000000
mean,0.943249,0.943249,0.943249,0.943249
std,0.056982,0.056982,0.056982,0.056982
min,0.746262,0.746262,0.746262,0.746262
25%,0.930348,0.930348,0.930348,0.930348
50%,0.954150,0.954150,0.954150,0.954150
75%,0.984551,0.984551,0.984551,0.984551
max,1.000000,1.000000,1.000000,1.000000


In [75]:
# Print mean f1 score
print("F1 score mean: ", eval_metrics_test['F1'].mean())

F1 score mean:  0.943249245584


### 6. Improve your model
Use grid search to find better parameters. 

In [76]:
# Define mean f1 score to use in grid search scoring object
def define_mean_f1_score(real_labels, predicted_labels):
    """Mean F1 score for all classifiers.
    
    Parameters:
    real_labels (list): real labels
    predicted_labels (list): predicted labels
        
    Returns:
    Mean F1 score for all  classifiers.
    """
    
    f1_score_list = []
    for i in range(np.shape(predicted_labels)[1]):
        f1 = f1_score(np.array(real_labels)[:, i], predicted_labels[:, i], average='micro')
        f1_score_list.append(f1)
        
    f1_score_avg = np.mean(f1_score_list)
    return f1_score_avg

In [25]:
# Apply grid search
parameters = {'vect__min_df': [1, 5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[30, 40],
              'clf__estimator__max_depth': [5, 10],
              'clf__estimator__n_jobs': [-1]}

scorer = make_scorer(define_mean_f1_score)
cv = GridSearchCV(pipeline, param_grid = parameters, scoring = scorer, n_jobs=-1, verbose = 48)

# Find best parameters
np.random.seed(777)
model_tuned = cv.fit(X_train, y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] clf__estimator__max_depth=5, clf__estimator__n_estimators=30, clf__estimator__n_jobs=-1, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__max_depth=5, clf__estimator__n_estimators=30, clf__estimator__n_jobs=-1, tfidf__use_idf=True, vect__min_df=1, score=0.9279108602041162, total=  26.6s
[CV] clf__estimator__max_depth=5, clf__estimator__n_estimators=30, clf__estimator__n_jobs=-1, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   42.2s remaining:    0.0s


[CV]  clf__estimator__max_depth=5, clf__estimator__n_estimators=30, clf__estimator__n_jobs=-1, tfidf__use_idf=True, vect__min_df=1, score=0.9281778727155596, total=  26.6s
[CV] clf__estimator__max_depth=5, clf__estimator__n_estimators=30, clf__estimator__n_jobs=-1, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  1.4min remaining:    0.0s


[CV]  clf__estimator__max_depth=5, clf__estimator__n_estimators=30, clf__estimator__n_jobs=-1, tfidf__use_idf=True, vect__min_df=1, score=0.9270081036178076, total=  26.7s
[CV] clf__estimator__max_depth=5, clf__estimator__n_estimators=30, clf__estimator__n_jobs=-1, tfidf__use_idf=True, vect__min_df=5 


...

[Parallel(n_jobs=-1)]: Done  46 out of  46 | elapsed: 34.6min remaining:    0.0s


[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=40, clf__estimator__n_jobs=-1, tfidf__use_idf=False, vect__min_df=5, score=0.9311955379242531, total=  31.8s
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=40, clf__estimator__n_jobs=-1, tfidf__use_idf=False, vect__min_df=5 


[Parallel(n_jobs=-1)]: Done  47 out of  47 | elapsed: 35.3min remaining:    0.0s


[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=40, clf__estimator__n_jobs=-1, tfidf__use_idf=False, vect__min_df=5, score=0.9304623130912419, total=  31.5s


[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed: 36.1min finished


In [26]:
# Grid search results
model_tuned.cv_results_

{'mean_fit_time': array([ 18.70833325,  19.11273869,  18.69748163,  18.96236173,
         20.14776095,  20.60505168,  20.09264199,  20.51008026,
         20.72041702,  21.30041512,  20.55255739,  21.4098165 ,
         24.65722386,  23.72671127,  22.70756364,  23.4592363 ]),
 'std_fit_time': array([ 0.05383641,  0.03279981,  0.0257381 ,  0.05521193,  0.07996026,
         0.06964401,  0.03995081,  0.08683638,  0.05736445,  0.07069808,
         0.00314361,  0.54278083,  0.45929885,  0.11693276,  0.06654176,
         0.11676999]),
 'mean_score_time': array([  7.91887975,   7.80269472,   7.95736233,   7.78974342,
          8.32372761,   7.9629395 ,   8.23606928,   7.98725828,
          7.93791548,   7.88548303,   8.00630569,  10.5992469 ,
          8.66427644,   8.13828786,   8.23225069,   8.15559864]),
 'std_score_time': array([ 0.00904158,  0.01449524,  0.0352506 ,  0.03427841,  0.17685826,
         0.00747904,  0.06967702,  0.03350936,  0.02350144,  0.03433817,
         0.04968413,  3.78

In [27]:
# Mean score for the best model
np.max(model_tuned.cv_results_['mean_test_score'])

0.93091157223748

In [28]:
# Parameters for the best model
model_tuned.best_params_

{'clf__estimator__max_depth': 10,
 'clf__estimator__n_estimators': 40,
 'clf__estimator__n_jobs': -1,
 'tfidf__use_idf': False,
 'vect__min_df': 5}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [29]:
# Evaluation metrics for test set
tuned_pred_test = model_tuned.predict(X_test)

eval_metrics_tuned = eval_metrics(np.array(y_test), tuned_pred_test, col_names)

In [30]:
eval_metrics_tuned

,Accuracy,Precision,Recall,F1
related,0.764876,0.764876,0.764876,0.764876
request,0.822246,0.822246,0.822246,0.822246
offer,0.994660,0.994660,0.994660,0.994660
aid_related,0.649832,0.649832,0.649832,0.649832
medical_help,0.919744,0.919744,0.919744,0.919744
medical_products,0.946750,0.946750,0.946750,0.946750
search_and_rescue,0.970400,0.970400,0.970400,0.970400
security,0.981538,0.981538,0.981538,0.981538
military,0.966738,0.966738,0.966738,0.966738
child_alone,1.000000,1.000000,1.000000,1.000000


In [31]:
eval_metrics_tuned.describe()

,Accuracy,Precision,Recall,F1
count,36.000000,36.000000,36.000000,36.000000
mean,0.926911,0.926911,0.926911,0.926911
std,0.081614,0.081614,0.081614,0.081614
min,0.649832,0.649832,0.649832,0.649832
25%,0.910665,0.910665,0.910665,0.910665
50%,0.950793,0.950793,0.950793,0.950793
75%,0.982530,0.982530,0.982530,0.982530
max,1.000000,1.000000,1.000000,1.000000


In [32]:
# Print mean f1 score
print("F1 score mean: ", eval_metrics_tuned['F1'].mean())

F1 score mean:  0.92691062286


We got lower mean f1 score than it was before without model tuning.

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [53]:
%%time
# Testing a pure decision tree classifier
moc = MultiOutputClassifier(DecisionTreeClassifier())

pipeline_moc = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', moc)
    ])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 777)
pipeline_moc.fit(X_train.as_matrix(), y_train.as_matrix())
y_pred_moc = pipeline_moc.predict(X_test)

CPU times: user 6min 2s, sys: 800 ms, total: 6min 3s
Wall time: 6min 5s


In [54]:
eval_metrics_moc = eval_metrics(np.array(y_test), y_pred_moc, col_names)

In [55]:
eval_metrics_moc

,Accuracy,Precision,Recall,F1
related,0.777846,0.777846,0.777846,0.777846
request,0.852609,0.852609,0.852609,0.852609
offer,0.991303,0.991303,0.991303,0.991303
aid_related,0.709643,0.709643,0.709643,0.709643
medical_help,0.906164,0.906164,0.906164,0.906164
medical_products,0.940037,0.940037,0.940037,0.940037
search_and_rescue,0.962313,0.962313,0.962313,0.962313
security,0.970400,0.970400,0.970400,0.970400
military,0.963229,0.963229,0.963229,0.963229
child_alone,1.000000,1.000000,1.000000,1.000000


In [56]:
eval_metrics_moc.describe()

,Accuracy,Precision,Recall,F1
count,36.000000,36.000000,36.000000,36.000000
mean,0.933183,0.933183,0.933183,0.933183
std,0.066522,0.066522,0.066522,0.066522
min,0.709643,0.709643,0.709643,0.709643
25%,0.926800,0.926800,0.926800,0.926800
50%,0.954837,0.954837,0.954837,0.954837
75%,0.979936,0.979936,0.979936,0.979936
max,1.000000,1.000000,1.000000,1.000000


In [57]:
# Print mean f1 score
print("F1 score mean: ", eval_metrics_moc['F1'].mean())

F1 score mean:  0.933183297732


### 9. Export your model as a pickle file

I exported the first model because it has higher average f1 score than the tuned model and the model with decision tree classifier.

In [78]:
pickle.dump(model_initial, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.